In [41]:
import numpy as np
import pandas as pd
import pandas_profiling
sort_by_importance = lambda x: x[1]

FEATURES_FILE = "./all_stations_evts_age.csv"

In [51]:
df=pd.read_csv(FEATURES_FILE)

df["Daily Weather"] = pd.Categorical(df["Daily Weather"]).codes
df["Daily Weather (Past)"] = pd.Categorical(df["Daily Weather (Past)"]).codes
df["Season"] = pd.Categorical(df["Season"]).codes
del df["Unnamed: 0"]
del df["Station ID"]
del df["Year"]

Y = df["Rented Bikes (Future)"].values
Y = Y.tolist()
del df["Rented Bikes (Future)"]
X = df.values

df=pd.read_csv(FEATURES_FILE)

df["Daily Weather"] = pd.Categorical(df["Daily Weather"]).codes
df["Daily Weather (Past)"] = pd.Categorical(df["Daily Weather (Past)"]).codes
df["Season"] = pd.Categorical(df["Season"]).codes
del df["Unnamed: 0"]
del df["Station ID"]
del df["Year"]

def replace_default(x):
    if x > 10:
        return float('nan')
    else:
        return x
df["nearestEvent"] = df["nearestEvent"].map(replace_default)
df["nearestEvent (Future)"] = df["nearestEvent (Future)"].map(replace_default)
df.head()

,Month,Day of Month,Day of Year,Day of Week,Season,Holiday,Daily Weather,Daily Weather (Past),Humidity,Humidity (Past),...,25-34M,25-34F,35-44M,35-44F,45-54M,45-54F,55-64M,55-64F,>65M,>65F
0,1,4,4,7,3,False,2,-1,0.94,NaN,...,30408,29870,18491,16158,13546,12929,8060,8695,9033,10977
1,1,5,5,1,3,False,3,2,0.88,0.94,...,30408,29870,18491,16158,13546,12929,8060,8695,9033,10977
2,1,6,6,2,3,False,3,3,0.86,0.88,...,30408,29870,18491,16158,13546,12929,8060,8695,9033,10977
3,1,7,7,3,3,False,4,3,0.86,0.86,...,30408,29870,18491,16158,13546,12929,8060,8695,9033,10977
4,1,8,8,4,3,False,5,4,0.87,0.86,...,30408,29870,18491,16158,13546,12929,8060,8695,9033,10977


In [52]:
profile = pandas_profiling.ProfileReport(df)
profile

/opt/anaconda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/opt/anaconda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/opt/anaconda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))
/opt/anaconda/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


Number of variables,34
Number of observations,139320
Total Missing (%),0.5%
Total size in memory,32.4 MiB
Average record size in memory,244.0 B
Numeric,17
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,15


In [53]:
profile.to_file(outputfile="pandas_profiling.html")

# Feature selection

See <https://hub.packtpub.com/4-ways-implement-feature-selection-python-machine-learning/>

## Univariate selection

In [49]:
import numpy
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)
#features = fit.transform(X)

numpy.set_printoptions(precision=3)
print("Higher is better")
for i in sorted(zip(df.columns,fit.scores_), key=sort_by_importance):
    print(i)

ValueError: could not convert string to float: 'Islington'

## Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
rfe = RFE(model, 3)
fit = rfe.fit(X, Y)

print("Num Features: %d"% fit.n_features_)

print("Lower is better")
print(("name","selected","ranking"))

for i in sorted(zip(df.columns, fit.ranking_, fit.support_), key=sort_by_importance):
    print(i)

## With random features

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

feat = df.keys()
feat_labels = feat.get_values()
train = df.values

Xtrain = train[:,0:13]
ytrain = train[:,13]

trees = 250
max_feat = 7
max_depth = 30
min_sample = 2
clf = RandomForestClassifier(n_estimators=trees, max_features=max_feat, max_depth=max_depth, min_samples_split= min_sample, random_state=0, n_jobs=-1)
clf.fit(Xtrain, ytrain.tolist())

print("Higher is better")
for feature in sorted(zip(feat_labels, clf.feature_importances_), key=sort_by_importance):
    print(feature)